In [1]:
print("Hello World")

Hello World


# Anki Connect API Integration

In [2]:
import json
import urllib.request

def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}

def invoke(action, debug=False, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    if debug: 
        print(requestJson)
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if debug: 
        print(response)
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

above is from https://foosoft.net/projects/anki-connect/

also: https://github.com/FooSoft/anki-connect/blob/master/plugin/__init__.py


Want: move "to learn" or "easy" to front of queue, or set specific interval to skip "learning" phase

see: `setSpecificValueOfCard`

important card fields: 
- type=2 (review)
- queue=2 (review)
- due or ivl ???

# Using API

### Experiments

In [3]:
# Get Deck Names
result = invoke('deckNames')
print('got list of decks: {}'.format(result))
# Error if anki closed: URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>

got list of decks: ['-exp', '-exp::anki-connect-test', '-exp::kanji-font-test', '...', 'backlog', 'nhg', 'nhg::1kanji', 'nhg::1kanji::common', 'nhg::1kanji::topic', 'nhg::2bunpou', 'nhg::3vocab', 'nhg::3vocab::auto-unsorted', 'nhg::3vocab::common', 'nhg::3vocab::topic', 'nhg::3vocab::unneeded', 'nhg::3vocab::Vocab Backlog']


In [4]:
# findCards is identical to guiBrowse (meaning we can use gui queries here)
# query = "deck:nhg Kanji:日"
# query = "deck:nhg::1kanji"

# query = "deck:nhg::3vocab"
query = "deck:-exp::anki-connect-test"
search_result = invoke("findCards", query=query)

In [5]:
# cardsInfo
# cards = search_result
cards = [1452224262134]
card_info = invoke("cardsInfo", cards=cards)
# takes 5 seconds for KANJI deck, takes 15 seconds for VOCAB deck

In [6]:
card_info[0].keys()
# important: fields, interval, queue, due

dict_keys(['cardId', 'fields', 'fieldOrder', 'question', 'answer', 'modelName', 'ord', 'deckName', 'css', 'factor', 'interval', 'note', 'type', 'queue', 'due', 'reps', 'lapses', 'left', 'mod'])

In [7]:
# print(card_info)
print("cardId:", card_info[0]["cardId"])
print("interval:", card_info[0]["interval"])
print("queue:", card_info[0]["queue"])
print("type: ", card_info[0]["type"])
print("due:", card_info[0]["due"])
print("deckName:", card_info[0]["deckName"])
print("vocab:", card_info[0]["fields"]["Word"]["value"])
print("type: ", card_info[0].keys())
# print("kanji:", card_info[0]["fields"]["Kanji"]["value"])

cardId: 1452224262134
interval: 0
queue: 0
type:  0
due: 3781
deckName: nhg::3vocab::unneeded
vocab: 改める
type:  dict_keys(['cardId', 'fields', 'fieldOrder', 'question', 'answer', 'modelName', 'ord', 'deckName', 'css', 'factor', 'interval', 'note', 'type', 'queue', 'due', 'reps', 'lapses', 'left', 'mod'])


# new card workflow

### addNote 

In [8]:
from jisho_api.word import Word

In [9]:
def create_furigana(word, reading):
    return "{}[{}]".format(word, reading)


def create_meaning(meanings):
    meaning_field = []
    for definition in meanings:
        defs = []
        for word in definition.english_definitions:
            defs.append(word)
            # print(word)
        one_def = ", ".join(defs)
        meaning_field.append(one_def)
    return f" (*) ".join(meaning_field)

In [10]:
# deckName = "-exp::anki-connect-test"
DECK_NAME = "nhg::3vocab::auto-unsorted"
NOTE_MODEL_NAME = "2-Vocab"

class DuplicateNoteException(Exception):
    pass

def create_note(word, allow_same=True, deck_name=DECK_NAME, generate=True):
    """
    -4 - ConnectionError
    -3 - Not in Jisho
    -2 - No exact match
    -1 - unknown error
    0 - duplicate
    else: card_id integer
    """
    note = dict()
    note["deckName"] = deck_name
    note["modelName"] = NOTE_MODEL_NAME
    note["tags"] = ["autocreated"]


    print("Creating card for:", word)
    fields = {}
    if generate:
        try:
            response = Word.request(word)#, cache=True)
        except ConnectionError as ce:
            print("ConnectionError Jisho")
            # TODO: implement retry here
            return -4
        if not response:
            print("Not in Jisho")
            return -3
        
        # Exclude same meaning verbs diff kanji?
        ind = None
        if allow_same:
            for i in range(len(response.data)):
                entry = response.data[i]
                for j in range(len(entry.japanese)):
                    if word == entry.japanese[j].word:
                        ind = dict()
                        ind["i"] = i
                        ind["j"] = j
                        ind["Word"] = entry.japanese[j].word
                    elif word == entry.japanese[j].reading:
                        ind = dict()
                        ind["i"] = i
                        ind["j"] = j
                        ind["Word"] = entry.japanese[j].reading
                    # entry.japanese[j].reading
        else:
            return -1
        if not ind:
            print("No exact match")
            return -2

        # Fill in fields
        fields["Word"] = ind["Word"]
        fields["Furigana"] = create_furigana(
            response.data[ind["i"]].japanese[ind["j"]].word, 
            response.data[ind["i"]].japanese[ind["j"]].reading
        )

        meaning = create_meaning(response.data[ind["i"]].senses)
        print(meaning)
        fields["Meaning"] = meaning
    else:
        # Fill in word only
        fields["Word"] = word


    note["fields"] = fields

    # try/accept
    try:
        result = invoke("addNote", note=note)
    except Exception as e:
        if str(e) == "cannot create note because it is a duplicate":
            print("Cannot create note because it is duplicate")
            return 0
        else:
            raise

    return result


In [11]:
# result = create_note("覇権", deck_name="-exp::anki-connect-test")
# print(result) # result is card number
# 覇権 has 4 slugs

### Word -> cardId


In [12]:
query = "deck:-exp::anki-connect-test Word:じ"
search_result = invoke("findCards", query=query)

In [13]:
assert len(search_result) == 1
search_result[0]


1706417082027

In [14]:
def get_word_card_id(word):
    print("Finding card id for:", word)
    query = "Word:{}".format(word)
    search_result = invoke("findCards", query=query)
    # Should be no dupes...
    # assert len(search_result) == 1
    if len(search_result) == 0:
        # print("Skipping")
        return None
    return search_result[0]

In [15]:
word = "字"
# print(get_word_card_id(word))

### `setSpecificValueOfCard`

- note: reached unsupported action because it wasn't in the __init__ file
- it's in the documentation and the repo, but not in the default addon
- i will just add it to the init file in the addon dir itself
- https://github.com/FooSoft/anki-connect/commit/54a7105bf9da31dabe884a96464206a0fc87c58d
- https://github.com/FooSoft/anki-connect/blob/master/plugin/__init__.py



- `ivl` won't update if new, only in DB
- setting `ivl`, `type`, `queue`, changes the interval but does not fix the due date
- setting `due` has to be relative to collection creation time
- can we just set to learning (`1`) and a higher interval?
- easy interval just has to be set to our desired
- setting `due` to zero, `type` and `queue` to 2 (review), and interval to desired
    - we can get the card we want to show as "Due" TODAY
    - the next interval will be interval*factor set in the deck options group



https://github.com/ankidroid/Anki-Android/wiki/Database-Structure
```
    type            integer not null,
      -- 0=new, 1=learning, 2=review, 3=relearning
    queue           integer not null,
      -- -3=user buried(In scheduler 2),
      -- -2=sched buried (In scheduler 2), 
      -- -2=buried(In scheduler 1),
      -- -1=suspended,
      -- 0=new, 1=learning, 2=review (as for type)
      -- 3=in learning, next rev in at least a day after the previous review
      -- 4=preview
```

In [16]:
# Invoke cardsInfo

cards = [1706417082027, 1452224257032]
result = invoke("cardsInfo", cards=cards)

r = result[1]
print("Healthy Card 油:")
print("cardId:", r["cardId"])
print("interval:", r["interval"])
print("type:", r["type"])
print("queue:", r["queue"])
print("due", r["due"])
print("\n")

c = result[0]
print("Test Card じ:")
print("cardId:", c["cardId"])
print("interval:", c["interval"])
print("type:", c["type"])
print("queue:", c["queue"])
print("due", c["due"])

Healthy Card 油:
cardId: 1452224257032
interval: 599
type: 2
queue: 2
due 2897


Test Card じ:
cardId: 1706417082027
interval: 5
type: 2
queue: 2
due 2184


In [17]:
# Invoke setSpecificValueOfCard
card = 1706417082027
keys = ["ivl", "type", "queue", "due"]
newValues = [5, 2, 2, 1]

#result = invoke("setSpecificValueOfCard", card=card, keys=keys, newValues=newValues, warning_check=True, debug=True)
# print(result)

In [18]:
from datetime import timedelta, date
VOCAB_DECK_CREATION_DATE = date(2018, 5, 10)

def set_card_learned(card_id, ivl):
    keys = ["ivl", "type", "queue", "due"]

    delta = abs(date.today() - VOCAB_DECK_CREATION_DATE)
    due = ivl + delta.days
    due_date = VOCAB_DECK_CREATION_DATE + timedelta(days=due)
    newValues = [ivl, 2, 2, due]
    # ivl   -> how long between last and current review
    #       -> changes to time from current review to future review
    # type  -> ???? 2 is the correct one tho
    # queue -> ???? 2 is the correct one tho
    # due   -> days FROM deck creation to be due on (0 is deck creation day)
    
    

    result = invoke("setSpecificValueOfCard", card=card_id, keys=keys, newValues=newValues, warning_check=True)
    if result and result[0]: 
        return due_date
    return None

In [19]:
# set_card_learned(1706417082027, 5)


### add+set
`the final function`

learned card
```
cardId: 1714022120139
interval: 1
queue: 2
type:  2
due: 2177
vocab: 半熟
```

unlearned card
```
cardId: 1706417082027
interval: 0
queue: 0
type:  0
due: 27940
vocab: じ
```



In [20]:
SKIP_DECK = "nhg::3vocab::unneeded"

import time
def auto_add_and_set(word, ivl, force_schedule=False, skip_schedule=False, skip_unneeded=True, generate=True):
    """
    None is failed!

    -3 "Not in Jisho"
    -2 "No exact match"
    -1 unknown error, 
    0 date given 
    1 skipping
    2 skipping scheduling - alr in deck
    3 skipping scheduling - in unneeded
    """
    print("--START-- auto_add_and_set -- ", word)
    if not word:
        return 1
    
    card_id = get_word_card_id(word)
    if not card_id:
        card_id = create_note(word, generate=generate)
    if card_id in {-1, -4}:
        return -1
    elif card_id in {-2}:
        return -2
    elif card_id in {-3}:
        return -3
        
    print(card_id)
    if skip_schedule:
        return 1
    # check_card_unlearned
    if not force_schedule:
        for i in range(5):
            while True:
                try: 
                    card_info = invoke("cardsInfo", cards=[card_id])
                    if card_info[0]["deckName"] == SKIP_DECK:
                        print("Skipping scheduling. Not needed")
                        return 3
                    if card_info[0]["type"] or card_info[0]["queue"]:
                        print("Skipping scheduling. Not a new card")
                        print("Card Interval:", card_info[0]["interval"])
                        return 2
                except KeyError as e:
                    print("Key 'type' doesn't exist yet")
                    return -1
                    # time.sleep(3)
                    # continue
                break

    result = set_card_learned(card_id, ivl)
    if result:
        return 0
    else:
        return -1

In [21]:
word = "拾い"
ivl = 1
# result = auto_add_and_set(word, 1)

### batch

91 words took 6m43.8s (first run)

In [22]:
def batch_add(words, ivl=-1, skip_schedule=True, force_schedule=True, generate=True):
    words_result_map = dict()
    for i in range(len(words)):
        print(i)
        word = words[i]
        try:
            result = auto_add_and_set(
                word, 
                ivl, 
                skip_schedule=skip_schedule,
                force_schedule=force_schedule,
                generate=False,
            )
            words_result_map[word] = result
        except Exception as e:
            print(e)
            words_result_map[word] = -1
        finally:
            print(f"{word}: {words_result_map[word]}")
            print()
    return words_result_map

# Unlearned workflows

### Get Unlearned Kanji Workflow

In [23]:
WORD_LIST = "other/highlights-text.txt"

In [24]:
# https://stackoverflow.com/questions/33338713/filtering-out-all-non-kanji-characters-in-a-text-with-python-3
# filter out others...

def is_not_hiragana_or_katakana(char):
    """
    Check if a character is not hiragana or katakana
    *from ChatGPT

    # Example usage
    characters = "あいうえおアイウエオ漢字"
    filtered_characters = [char for char in characters if is_not_hiragana_or_katakana(char)]

    print("Original Characters:", characters)
    print("Filtered Characters:", "".join(filtered_characters))
    """
    import re
    # Define regular expressions for Hiragana and Katakana
    hiragana_pattern = re.compile(r'[\u3041-\u3096ー]')  # Hiragana
    katakana_pattern = re.compile(r'[\u30A0-\u30FFー]')  # Katakana

    # Check if the character is not a Hiragana or Katakana character
    return not (hiragana_pattern.match(char) or katakana_pattern.match(char))

In [25]:
def get_kanji(word_list):
    # from gpt
    with open(word_list, encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f.readlines()]

    # set
    kanji_to_search = set()
    for line in lines:
        for char in line:
            # TODO (ebui): filter out all non kanji characters
            if is_not_hiragana_or_katakana(char):
                kanji_to_search.add(char)

    # Get Entire Deck (findCards -> cardsInfo) 7s
    query = "deck:nhg::1kanji"
    search_result = invoke("findCards", query=query)
    card_info = invoke("cardsInfo", cards=search_result)

    # Create Map
    cards = dict()
    failed = list()
    for card in card_info:
        try:
            kanji = card["fields"]["Kanji"]["value"]
            cards[kanji] = card["interval"]
        except KeyError as ke:
            failed.append(card)


    # Search
    to_learn = set()
    learned = set()
    not_found = set()
    for kanji in kanji_to_search:
        try:
            if cards[kanji] == 0:
                to_learn.add(kanji)
            elif cards[kanji] > 0:
                learned.add(kanji)
        except Exception:
            not_found.add(kanji)

    print("--")
    print("To learn:\t", len(to_learn), to_learn)
    print("Learned:\t", len(learned), learned)
    print("Not Found:\t", len(not_found), not_found)

    return to_learn, learned
# 8s

In [26]:
kanji_unlearned, kanji_learned = get_kanji(WORD_LIST)

--
To learn:	 0 set()
Learned:	 206 {'援', '基', '日', '乏', '集', '引', '乗', '一', '活', '札', '余', '律', '客', '性', '茂', '文', '訪', '実', '豊', '力', '立', '海', '首', '掲', '女', '部', '絵', '化', '本', '九', '向', '議', '々', '州', '考', '茶', '生', '気', '左', '際', '廃', '後', '源', '様', '核', '益', '含', '明', '小', '大', '済', '内', '初', '始', '国', '民', '半', '揮', '論', '市', '再', '長', '策', '緯', '設', '巡', '全', '定', '加', '中', '道', '稼', '令', '年', '課', '光', '方', '関', '可', '魅', '能', '反', '情', '統', '普', '挙', '合', '有', '土', '要', '交', '支', '練', '省', '頼', '住', '期', '的', '境', '体', '指', '世', '開', '観', '好', '増', '意', '街', '自', '横', '幌', '薄', '広', '来', '果', '枢', '減', '点', '題', '助', '効', '府', '働', '治', '北', '村', '度', '新', '配', '識', '然', '地', '誇', '税', '口', '何', '時', '濁', '庁', '会', '略', '描', '在', '探', '創', '町', '切', '辺', '従', '展', '像', '伝', '経', '多', '破', '金', '今', '戦', '納', '遍', '賭', '人', '石', '発', '現', '岡', '得', '福', '機', '政', '目', '保', '都', '将', '通', '各', '荒', '直', '付', '残', '進', '事', '資', '所', '右', '相', '同', '捨', '招', '導', '周', '例', '重'

### Get Unlearned Words Workflow

In [27]:
def get_words(word_list):
    # from gpt
    with open(word_list, encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f.readlines()]

    # set
    words_to_search = set(lines)

    # Get Entire Deck (findCards -> cardsInfo) 7s
    query = "deck:nhg::3vocab"
    search_result = invoke("findCards", query=query)
    card_info = invoke("cardsInfo", cards=search_result)

    # Create Map
    cards = dict()
    failed = list()

    to_skip = set()
    for card in card_info:
        try:
            vocab = card["fields"]["Word"]["value"]
            if card["deckName"] == SKIP_DECK:
                # to_skip.add(vocab)
                cards[vocab] = -100000
            else:
                cards[vocab] = card["interval"]
        except KeyError as ke:
            failed.append(card)

    # Info
    to_learn = set()
    learned = set()
    not_found = set()
    for word in words_to_search:
        try:
            if cards[word] == 0:
                to_learn.add(word)
            elif cards[word] > 0:
                learned.add(word)
            elif cards[word] == -100000:
                to_skip.add(word)
        except Exception:
            not_found.add(word)

    print("--")
    print("to learn:\t", len(to_learn), to_learn)
    print("learned:\t", len(learned), learned)
    print("to add:\t", len(not_found), not_found)
    print("to skip:\t", len(to_skip), to_skip)
    print("failed:\t",  len(failed), failed)
    # return to_learn.union(not_found), learned, to_skip, failed
    return to_learn.union(not_found), learned, to_skip

# 16s

In [28]:
file = "other/girlsbandcry1-cleaned.txt"
with open(file, encoding='utf-8') as f:
    lines = [line.rstrip('\n') for line in f.readlines()]
print(len(set(lines)))
# words_unlearned, words_learned, to_add, to_skip, failed = get_words(file)
# print(len(words_unlearned) + len(words_learned) + len(to_add) + len(to_skip) + len(failed))
# --
# words_unlearned, words_learned, to_skip = get_words(file)
# print(len(words_unlearned) + len(words_learned) + len(to_skip))


574


### combined workflow: Get words with known kanji and words with unknown kanji

In [29]:
# kanji_unlearned, kanji_learned
# words_unlearned, words_learned

# word true/false, kanji true/false

"""
wf_kt = set() # Easy Words
wf_kf = set() # Low Prio Words
wt_kf = set() # High Prio Kanji
wt_kt = set() # Known


for word in words_unlearned:
    known = True
    for char in word:
        if char in kanji_unlearned:
            wf_kf.add(word)
            known = False
            break
    if known:
        wf_kt.add(word)
for word in words_learned:
    known = True
    for char in word:
        if char in kanji_unlearned:
            wt_kf.add(word)
            known = False
            break
    if known:
        wt_kt.add(word)

# Warning: ignores words with kanji not found, doesn't separate words not found
print("wf_kt, Easy Words:", len(wf_kt), wf_kt)
print("wf_kf, Low Prio Word:", len(wf_kf), wf_kf)
print("wt_kf, High Prio Kanji:", len(wt_kf), wt_kf)
print("wt_kt, Known Words:", len(wt_kt), wt_kt)
"""

'\nwf_kt = set() # Easy Words\nwf_kf = set() # Low Prio Words\nwt_kf = set() # High Prio Kanji\nwt_kt = set() # Known\n\n\nfor word in words_unlearned:\n    known = True\n    for char in word:\n        if char in kanji_unlearned:\n            wf_kf.add(word)\n            known = False\n            break\n    if known:\n        wf_kt.add(word)\nfor word in words_learned:\n    known = True\n    for char in word:\n        if char in kanji_unlearned:\n            wt_kf.add(word)\n            known = False\n            break\n    if known:\n        wt_kt.add(word)\n\n# Warning: ignores words with kanji not found, doesn\'t separate words not found\nprint("wf_kt, Easy Words:", len(wf_kt), wf_kt)\nprint("wf_kf, Low Prio Word:", len(wf_kf), wf_kf)\nprint("wt_kf, High Prio Kanji:", len(wt_kf), wt_kf)\nprint("wt_kt, Known Words:", len(wt_kt), wt_kt)\n'

### analyze list

In [30]:
def analyze_list(word_list):
    kanji_unlearned, kanji_learned = get_kanji(word_list)
    words_unlearned, words_learned, to_skip = get_words(word_list)

    # word true/false, kanji true/false
    wf_kt = set() # Easy Words
    wf_kf = set() # Low Prio Words
    wt_kf = set() # High Prio Kanji
    wt_kt = set() # Known


    for word in words_unlearned:
        known = True
        for char in word:
            if char in kanji_unlearned:
                wf_kf.add(word)
                known = False
                break
        if known:
            wf_kt.add(word)
    for word in words_learned:
        known = True
        for char in word:
            if char in kanji_unlearned:
                wt_kf.add(word)
                known = False
                break
        if known:
            wt_kt.add(word)

    # Warning: ignores words with kanji not found, doesn't separate words not found
    print("--")
    print("wf_kt,\tEasy Words:", len(wf_kt), sorted(wf_kt, key=len))
    print("wf_kf,\tLow Prio Word:", len(wf_kf), wf_kf)
    print("wt_kf,\tHigh Prio Kanji:", len(wt_kf), wt_kf)
    print("wt_kt,\tKnown Words:", len(wt_kt), sorted(wt_kt, key=len))
    print("UNNEEDED (SKIP): \t", len(to_skip), sorted(to_skip, key=len))
    print()
    # easy, low, kanji, known
    return  sorted(wf_kt, key=len), sorted(wf_kf, key=len)

In [31]:
myl = "other/girlsbandcry1-cleaned.txt"
# easy, low, kanji, known = analyze_list(myl)

# problems



### problems
Problems noticed within the autocreated cards:
- words that are definitely not important
    - maybe need a way to search up frequency for real? 
- compound words -> example 低血圧 vs 血圧 (i prio base words)
    - compound words can be in the meaning description
        致命
            致命的
            致命傷
- add sentence example?


1) should keep in auto-unsorted until reviewed once


-> make function to batch them
-> return a return code, 


EXAMPLE PROBLEM WORD: あいびき

# SPACE

###### EXECUTE ABOVE CELLS

In [32]:
print("FINISHED!")

FINISHED!


In [38]:
# EXECUTE ABOVE CELLS

In [2]:
# SCHEDULE 
word = "."
ivl = 7
result = auto_add_and_set(word, ivl, force_schedule=True)
print(result)

NameError: name 'auto_add_and_set' is not defined

In [57]:
# SKIP SCHEDULE
word = "."
ivl = 4
result = auto_add_and_set(word, ivl, skip_schedule=True)

--START-- auto_add_and_set --  湊
Finding card id for: 湊
1729815645393


In [34]:
###################################

...

In [192]:
# SCHEDULE IF NOT SCHEDULED - FIRST
word = [
"穿つ"
]
ivl = 1
result = auto_add_and_set(word[0], ivl)
result

--START-- auto_add_and_set --  穿つ
Finding card id for: 穿つ
Creating card for: 穿つ
to drill, to bore, to pierce, to pass through (*) to hit the mark, to get to the heart of (the matter), to be true to (nature) (*) to put on, to wear
1730080704693
Key 'type' doesn't exist yet


-1

In [194]:
# SCHEDULE IF NOT SCHEDULED - SECOND
word = [
"鼻孔"
]
ivl = 4
result = auto_add_and_set(word[0], ivl)
result

--START-- auto_add_and_set --  鼻孔
Finding card id for: 鼻孔
1730080937059


0

In [195]:
# SCHEDULE IF NOT SCHEDULED - THIRD
word = [
"鼻の穴"
]
ivl = 360
result = auto_add_and_set(word[0], ivl)
result

--START-- auto_add_and_set --  鼻の穴
Finding card id for: 鼻の穴
Creating card for: 鼻の穴
nostril
1730080962549


0

##### me analyse


- note that words with not found kanji are not added...
- add not found kanji and rerun
- to add: if autocreated, or added already, definitely schedule

In [34]:
# FILE_NAME = "other/hl-cleaned-imokano-1.txt"
# FILE_NAME = "parsed/highlights-renaisoudan-3.txt"
FILE_NAME = "parsed/subs-rezero-s3e2.txt"
easy, low = analyze_list(FILE_NAME)

--
To learn:	 4 {'呉', '嘗', '蒔', '瞑'}
Learned:	 441 {'用', '根', '邪', '僕', '一', '価', '札', '尻', '強', '吹', '巻', '静', '追', '次', '尊', '応', '楽', '惰', '鈴', '飽', '首', '部', '係', '園', '々', '気', '判', '明', '髪', '原', '遅', '終', '歌', '限', '覚', '長', '全', '到', '法', '申', '関', '取', '起', '能', '情', '理', '反', '前', '有', '弁', '的', '沸', '間', '異', '最', '来', '回', '称', '汚', '聖', '捕', '為', '頭', '配', '息', '洗', '郎', '救', '殺', '映', '時', '値', '絶', '会', '談', '探', '歓', '刀', '紹', '許', '従', '失', '紛', '替', '正', '敵', '臭', '想', '身', '行', '寄', '存', '現', '得', '焼', '頂', '縁', '不', '通', '直', '習', '飛', '悲', '知', '付', '心', '圧', '場', '苦', '死', '安', '話', '形', '走', '解', '供', '準', '二', '変', '引', '子', '毛', '恵', '近', '違', '鬼', '使', '郷', '恐', '断', '本', '必', '向', '考', '焦', '拗', '様', '吐', '制', '範', '謎', '剣', '由', '過', '続', '初', '止', '渉', '権', '派', '男', '泣', '中', '嫌', '押', '施', '抱', '攻', '合', '要', '詞', '痛', '上', '居', '味', '謝', '聴', '腹', '利', '約', '言', '街', '欲', '果', '成', '撃', '笑', '飲', '作', '効', '置', '闇', '皆', '冗', '誘', '凄', '空', '信', '肝', '介'

##### batch

In [153]:
%%capture cap
# cap.show() to see stdout

# add `easy`, then `low`
ivl=7
# skip=False
# false for easy, true for low
# force schedule on (use skip as toggle instead)
results = batch_add(
    easy+low, 
    ivl=ivl, 
    skip_schedule=False, 
    force_schedule=False,
)

##### batch results

142 -> 37min, 156 -> 32min, 128 -> 24min, 102 -> 23min

Ideas to speed this up (parallelize):
1) create the cards first if needed, then schedule 
2) have a queue for the card generation and EDIT the cards only (could add an option to skip generation entirely and just create front cards)
3) print progress in batches for both the card creation queue and card generation queue

In [154]:
results
len(results)
# results_sorted = {y: x for x, y in results.items()}
inv_map = {}
for k, v in results.items():
    inv_map[v] = inv_map.get(v, []) + [k]

total = 0
for k, values in inv_map.items():
    total += len(values)
print(total)

inv_map

102


{0: ['癪',
  '犬歯',
  '反復',
  '湯気',
  '金魚',
  '連結',
  '沿道',
  '皆無',
  '険悪',
  '誰彼',
  '氷柱',
  '合掌',
  '配置',
  '神楽',
  '類型',
  '贅肉',
  '滞り',
  '薄手',
  '戦果',
  '戯れ',
  '下地',
  '標語',
  '抵触',
  '露天',
  '並木',
  '近辺',
  '点呼',
  '裏目',
  '足下',
  '四隅',
  '花柄',
  '目蓋',
  '凶報',
  '語尾',
  '快挙',
  '眉間',
  '凍死',
  '言霊',
  '私情',
  '境内',
  '海鮮',
  'ヤワ',
  '大小',
  '安パイ',
  '着直す',
  '甲子園',
  'パンク',
  '平謝り',
  '持久力',
  '補助輪',
  'ぽつり',
  'オセロ',
  '連ねる',
  'ちろり',
  '耳年増',
  '一押し',
  '有識者',
  '滑走路',
  'つむる',
  'すっと',
  'お通夜',
  '冷や水',
  'ゆさゆさ',
  'がやがや',
  'ハイエナ',
  '相まって',
  'くまなく',
  'ちまちま',
  'ぷるぷる',
  'ごそごそ',
  'サバサバ',
  '福神漬け',
  'モリモリ',
  '独占禁止法',
  'ノースリーブ',
  'ごにょごにょ',
  'レジャーシート',
  '鳩',
  '茜色'],
 -3: ['円札'],
 -2: ['弁女',
  '素っ気',
  '細さも',
  '敷いて',
  'ノロケ',
  'べしべし',
  'そむけた',
  '鼻白んだ',
  'アリアリ',
  '名残惜し',
  '窘められた',
  'メモを手に',
  'しょげている',
  'ぷにぷにだね',
  '穿いて'],
 -1: ['帳尻', '姿見', '御の字', 'お役御免', '余所行き', 'クレーマー', '回りくどい']}

In [155]:
...

Ellipsis